# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
!pip install xlsxwriter

     |████████████████████████████████| 153kB 5.7MB/s 


In [2]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [5]:
stocks = pd.read_csv('sp_500_stocks.csv')
#from secrets import IEX_CLOUD_API_TOKEN
import secrets
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
print(IEX_CLOUD_API_TOKEN)

Tpk_059b97af715d417d9f49f50b51b1c448


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [6]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 98769480,
 'calculationPrice': 'close',
 'change': 1.5,
 'changePercent': 0.0125,
 'close': 129.41,
 'closeSource': 'fcaifoli',
 'closeTime': 1632303633686,
 'companyName': 'Apple Inc',
 'delayedPrice': 127.461,
 'delayedPriceTime': 1649132798350,
 'extendedChange': -0.24,
 'extendedChangePercent': -0.00187,
 'extendedPrice': 126.39,
 'extendedPriceTime': 1641444928328,
 'high': 130.44,
 'highSource': ' ec 1y5ieutpdnilr edame',
 'highTime': 1687532650670,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 128.51,
 'iexCloseTime': 1659630278704,
 'iexLastUpdated': 1643256111952,
 'iexMarketPercent': 0.014933297633103877,
 'iexOpen': 124.11,
 'iexOpenTime': 1617373773391,
 'iexRealtimePrice': 129.09,
 'iexRealtimeSize': 102,
 'iexVolume': 1309098,
 'isUSMarketOpen': False,
 'lastTradeTime': 1670794338843,
 'latestPrice': 128.17,
 'latestSource': 'Close',
 'latestTime': 'December 7, 2020',
 'latestUpdate': 1681021875433,
 'latestVolum

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [7]:
pe_ratio = data['peRatio']
pe_ratio

37.87

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [8]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [10]:
symbol_strings[0]

'A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA'

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [11]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,115.90,53.01,N/A
1,AAL,17.77,-1.26,N/A
2,AAP,158.92,22.59,N/A
3,AAPL,127.86,39.39,N/A
4,ABBV,107.06,23.37,N/A
...,...,...,...,...
500,YUM,105.47,31.53,N/A
501,ZBH,149.59,-204.9,N/A
502,ZBRA,385.01,43.97,N/A
503,ZION,44.90,18.62,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [12]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [13]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [14]:
portfolio_input()

Enter the value of your portfolio:10000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [18]:
final_dataframe['Price'][0:5]

0     34.17
1     20.26
2    583.94
3     24.00
4     15.59
Name: Price, dtype: float64

In [20]:
final_dataframe.loc[0:5, 'Price']

0     34.17
1     20.26
2    583.94
3     24.00
4     15.59
5    106.03
Name: Price, dtype: float64

In [ ]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [26]:
data['AAPL']['quote']['peRatio']

39.31

In [21]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [28]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [29]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,215.70,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,238.63,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,N/A,10.03,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,25.91,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.83,N/A,14.61,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,57.60,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,30.53,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,134.32,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
194,FTI,9.93,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1833.95,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [30]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [31]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [32]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.857426
1      0.140594
2      0.438614
3      0.722772
4      0.447525
         ...   
500    0.653465
501    0.019802
502    0.772277
503    0.362376
504    0.833663
Name: PE Percentile, Length: 505, dtype: object
0       0.710891
1      0.0435644
2            0.4
3       0.962376
4       0.849505
         ...    
500     0.039604
501     0.372277
502     0.825743
503     0.110891
504     0.942574
Name: PB Percentile, Length: 505, dtype: object
0        0.79802
1      0.0316832
2       0.165347
3       0.835644
4       0.683168
         ...    
500     0.762376
501     0.624752
502     0.673267
503      0.40396
504     0.932673
Name: PS Percentile, Length: 505, dtype: object
0       0.857426
1      0.0277228
2       0.259406
3        0.79802
4        0.49901
         ...    
500     0.726733
501     0.774257
502     0.829703
503     0.233663
504     0.855446
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0       0.807921
1      0.0554455
2       0.124752
3       0.946

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,115.80,N/A,53.42,0.857426,7.100,0.710891,6.8000,0.79802,34.435789,0.857426,13.405252,0.807921,N/A
1,AAL,17.92,N/A,-1.25,0.140594,-1.890,0.0435644,0.4274,0.0316832,-6.090046,0.0277228,1.377571,0.0554455,N/A
2,AAP,154.59,N/A,23.05,0.438614,2.810,0.4,1.0900,0.165347,10.691350,0.259406,2.478275,0.124752,N/A
3,AAPL,129.07,N/A,38.67,0.722772,32.800,0.962376,7.6700,0.835644,27.221272,0.79802,20.301916,0.946535,N/A
4,ABBV,106.53,N/A,23.51,0.447525,12.280,0.849505,4.7000,0.683168,16.098280,0.49901,9.526949,0.657426,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.01,N/A,32.72,0.653465,-4.100,0.039604,6.0500,0.762376,23.123293,0.726733,9.697268,0.663366,N/A
501,ZBH,151.30,N/A,-199.50,0.019802,2.610,0.372277,4.4700,0.624752,25.217344,0.774257,7.602997,0.49703,N/A
502,ZBRA,374.93,N/A,43.04,0.772277,10.610,0.825743,4.6000,0.673267,29.558940,0.829703,11.452631,0.736634,N/A
503,ZION,42.99,N/A,18.53,0.362376,1.017,0.110891,2.4100,0.40396,10.481137,0.233663,2.796838,0.142574,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [34]:
for row in rv_dataframe.index[:5]:
  print(row)

0
1
2
3
4


In [36]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,115.800,N/A,53.420000,0.857426,7.100000,0.710891,6.800000,0.79802,34.435789,0.857426,13.405252,0.807921,0.806337
1,NTRS,94.940,N/A,16.120000,0.30495,1.890000,0.259406,3.170000,0.518812,11.538231,0.29901,3.221158,0.194059,0.315248
2,NTAP,61.830,N/A,17.580000,0.342574,48.420000,0.976238,2.500000,0.416832,10.710650,0.261386,3.363825,0.20396,0.440198
3,NSC,248.510,N/A,32.020000,0.633663,4.430000,0.552475,6.740000,0.792079,17.481169,0.544554,7.851955,0.522772,0.609109
4,NRG,34.930,N/A,2.120000,0.150495,4.810000,0.586139,0.888200,0.10495,7.125188,0.0930693,4.959494,0.293069,0.245545
5,NOW,549.580,N/A,150.450000,0.980198,43.470000,0.968317,27.060000,0.99604,211.930121,0.99802,33.114702,0.994059,0.987327
6,NOV,14.650,N/A,-2.130000,0.130693,1.008300,0.108911,0.806900,0.0831683,24297.159298,1,6.937209,0.437624,0.352079
7,NOC,306.000,N/A,20.710000,0.411881,4.960000,0.594059,1.450000,0.229703,11.147226,0.279208,7.971543,0.530693,0.409109
8,NLSN,17.730,N/A,-41.640000,0.0514851,3.150000,0.445545,0.982000,0.122772,8.423787,0.142574,3.900990,0.231683,0.198812
9,NLOK,19.780,N/A,3.850000,0.152475,-25.410000,0.0158416,6.290000,0.776238,16.452749,0.512871,8.247599,0.548515,0.401188


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [35]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [37]:
portfolio_input()

Enter the value of your portfolio:10000000


In [ ]:
for i in range(0, len(rv_dataframe['Ticker'])-1):
  print(i)

In [39]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,115.800,1727,53.420000,0.857426,7.100000,0.710891,6.800000,0.79802,34.435789,0.857426,13.405252,0.807921,0.806337
1,NTRS,94.940,2106,16.120000,0.30495,1.890000,0.259406,3.170000,0.518812,11.538231,0.29901,3.221158,0.194059,0.315248
2,NTAP,61.830,3234,17.580000,0.342574,48.420000,0.976238,2.500000,0.416832,10.710650,0.261386,3.363825,0.20396,0.440198
3,NSC,248.510,804,32.020000,0.633663,4.430000,0.552475,6.740000,0.792079,17.481169,0.544554,7.851955,0.522772,0.609109
4,NRG,34.930,5725,2.120000,0.150495,4.810000,0.586139,0.888200,0.10495,7.125188,0.0930693,4.959494,0.293069,0.245545
5,NOW,549.580,363,150.450000,0.980198,43.470000,0.968317,27.060000,0.99604,211.930121,0.99802,33.114702,0.994059,0.987327
6,NOV,14.650,13651,-2.130000,0.130693,1.008300,0.108911,0.806900,0.0831683,24297.159298,1,6.937209,0.437624,0.352079
7,NOC,306.000,653,20.710000,0.411881,4.960000,0.594059,1.450000,0.229703,11.147226,0.279208,7.971543,0.530693,0.409109
8,NLSN,17.730,11280,-41.640000,0.0514851,3.150000,0.445545,0.982000,0.122772,8.423787,0.142574,3.900990,0.231683,0.198812
9,NLOK,19.780,10111,3.850000,0.152475,-25.410000,0.0158416,6.290000,0.776238,16.452749,0.512871,8.247599,0.548515,0.401188


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [42]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [43]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [44]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [45]:
writer.save()